---
# **IMPORT**
---

In [2]:
import os
import io
from embeding import *
from vector_store import *
import csv
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt

/home/alex/miniconda3/envs/ai_challenge/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
qdrant_url = "http://localhost:6333"
token = ""

vector_space = QdrantVectorSpace(
    qdrant_url=qdrant_url,
    token=token,
    collection_name="run",
    snapshot=True
)
vector_space.load_snapshot("../weights/snapshot/qdrant_snapshot.snapshot")

/home/alex/miniconda3/envs/ai_challenge/lib/python3.10/site-packages/qdrant_client/qdrant_remote.py:130: UserWarning: Api key is used with an insecure connection.
  warnings.warn("Api key is used with an insecure connection.")


Snapshot uploaded successfully.
Response: {'result': True, 'status': 'ok', 'time': 10.745822849}


In [9]:
model = JinaTextEmbeding('../weights/jina-clip-v1')

---
# **SETTING BEFORE RUNNING**
---

In [10]:
# Folder path
folder_sum = "run"
folder_input = os.path.join(folder_sum, "english")
folder_output = os.path.join(folder_sum, "result")
folder_check = os.path.join(folder_sum, "check")

os.makedirs(folder_output, exist_ok=True)
os.makedirs(folder_check, exist_ok=True)

troll_video = ["L09_V009", "L06_V003"]

---
# **RUNNING**
---

### - **Note:** Have to run from repo temp_ai_flow

In [11]:
%cd ..

/home/alex/AI Challenge/temp_AI_flow


/home/alex/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


### - **Old structure**

In [6]:
# data = []
# file_names = os.listdir(folder_input)
# for each in file_names:
#     file_path = os.path.join(folder_input, each)
#     with open(file_path, 'r') as f:
#         data.append(f.read())

In [7]:
# result = model.inference(data, norm_embeds=True)

In [8]:
# images_solve = {}

# file_names = os.listdir(folder_input)
# for each_embedding, each_file in zip(result[0], file_names):
#     answer = vector_space.query_by_text(each_embedding, top_k=100)
#     images_solve[each_file] = []
#     # Result
#     file_output = os.path.join(folder_output, each_file)
#     file_output = file_output.replace(".txt", ".csv")

#     # Check
#     file_check = os.path.join(folder_check, each_file)
#     file_check = file_check.replace(".txt", ".csv")

#     with open(file_output, 'w', newline='') as csvfile:
#         csvwriter = csv.writer(csvfile)
        
#         for scored_point in answer:
#             video_id = scored_point.payload.get('video_id', '')
#             frame_id = scored_point.payload.get('frame_id', '')
#             images_solve[each_file].append((video_id, frame_id))
#             if video_id in troll_video:
#                 frame_id = str(int(frame_id) * 25 / 30)
#             csvwriter.writerow([video_id, round(float(frame_id))])

#     with open(file_check, 'w', newline='') as csvfile:
#         csvwriter = csv.writer(csvfile)
#         csvwriter.writerow(['Video ID', 'Frame ID', 'BTC Frame ID', 'Time'])
        
#         for scored_point in answer:
#             video_id = scored_point.payload.get('video_id', '')
#             frame_id = scored_point.payload.get('frame_id', '')

#             index_time = int(frame_id) / 25
#             minute = index_time // 60
#             second = index_time % 60
#             index_time = f"{int(minute)}min{second:.2f}s"

#             btc_frame_id = frame_id
#             if video_id in troll_video:
#                 btc_frame_id = int(frame_id) * 25 / 30
                
#             csvwriter.writerow([video_id, frame_id, btc_frame_id, index_time])

### - **New structure**

In [12]:
data = {}
question_folders = os.listdir(folder_input)
for each_folder in question_folders:
    question_folder_path = os.path.join(folder_input, each_folder)
    question_files = os.listdir(question_folder_path)
    data[each_folder] = []
    for each_file in question_files:
        file_path = os.path.join(question_folder_path, each_file)
        with open(file_path, 'r') as f:
            data[each_folder].append(f.read()) 

In [13]:
result = {}
for each_key in data.keys():
    result[each_key] = model.inference(data[each_key], norm_embeds=True)

In [14]:
for each_folder in data.keys():
    folder_output_path = os.path.join(folder_output, each_folder)
    folder_check_path = os.path.join(folder_check, each_folder)

    os.makedirs(folder_output_path, exist_ok=True)
    os.makedirs(folder_check_path, exist_ok=True)
    answer_for_each_query = []

    for embeddings in result[each_folder]: 
        for i, each_embedding in enumerate(embeddings):
            answer = vector_space.query_by_text(each_embedding, top_k=200)
            
            file_output = os.path.join(folder_output_path, f"{i}.csv")
            file_output_check = os.path.join(folder_check_path, f"{i}.csv")

            with open(file_output, 'w', newline='') as csvfile:
                csvwriter = csv.writer(csvfile)
                for scored_point in answer:
                    video_id = scored_point.payload.get('video_id', '')
                    frame_id = scored_point.payload.get('frame_id', '')
                    if video_id in troll_video:
                        frame_id = str(int(frame_id) * 25 / 30)
                    csvwriter.writerow([video_id, round(float(frame_id))])

            with open(file_output_check, 'w', newline='') as csvfile:
                csvwriter = csv.writer(csvfile)
                csvwriter.writerow(['Video ID', 'Frame ID', 'BTC Frame ID', 'Time', 'Score'])
                for scored_point in answer:
                    video_id = scored_point.payload.get('video_id', '')
                    frame_id = scored_point.payload.get('frame_id', '')
                    score = scored_point.score

                    index_time = int(frame_id) / 25
                    minute = index_time // 60
                    second = index_time % 60
                    index_time = f"{int(minute)}min{second:.2f}s"

                    btc_frame_id = frame_id
                    if video_id in troll_video:
                        btc_frame_id = int(int(frame_id) * 25 / 30)
                        
                    csvwriter.writerow([video_id, frame_id, btc_frame_id, index_time, score])

---
# **STATISTIC**
---

In [12]:
def process_rerank(df, frame_interval=200):
    most_video_id_df = df['Video ID'].value_counts().reset_index()
    most_video_id_df.columns = ['Video ID', 'Count']
    most_video_id_df = most_video_id_df.sort_values(by='Count', ascending=False)

    keyframe_distribution = []
    for video_id in df['Video ID'].unique():
        video_df = df[df['Video ID'] == video_id]

        video_df.loc[:, 'Frame Interval'] = (video_df['Frame ID'] // frame_interval) * frame_interval
        interval_count = video_df['Frame Interval'].value_counts().reset_index()
        interval_count.columns = ['Frame Interval', 'Count']
        interval_count = interval_count.sort_values(by='Count', ascending=False)

        for _, row in interval_count.iterrows():
            time = row['Frame Interval'] / 25
            min = time // 60
            second = time % 60
            keyframe_distribution.append({
                'Video ID': video_id,
                'Frame Interval': int(row['Frame Interval']),
                'Keyframe Count': int(row['Count']),
                'Time': f"{int(min)}min{int(second)}s"
            })
    most_frame_id_df = pd.DataFrame(keyframe_distribution)
    return most_video_id_df, most_frame_id_df

In [13]:
for each_folder in os.listdir(folder_check):
    folder_check_path = os.path.join(folder_check, each_folder)
    query_dataframe = []
    for each_file in os.listdir(folder_check_path):
        file_path = os.path.join(folder_check_path, each_file)
        df = pd.read_csv(file_path)
        query_dataframe.append(df)
    
    query_dataframe = pd.concat(query_dataframe)
    most_video_id, most_frame_id = process_rerank(query_dataframe)
    most_video_id.to_csv(os.path.join(folder_check_path, "most_video_id.csv"), index=False)
    most_frame_id.to_csv(os.path.join(folder_check_path, "most_frame_id.csv"), index=False)

/tmp/ipykernel_328047/880627434.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  video_df.loc[:, 'Frame Interval'] = (video_df['Frame ID'] // frame_interval) * frame_interval
/tmp/ipykernel_328047/880627434.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  video_df.loc[:, 'Frame Interval'] = (video_df['Frame ID'] // frame_interval) * frame_interval
/tmp/ipykernel_328047/880627434.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

---
# **ADD RESULT FOR QA**
---

### **- Add**

In [16]:
# Add column with the same value to the file
import random
input_file = "/mnt/c/Users/huyho/Downloads/pack2-groupA/Submission/Done/query-p2-3-qa.csv"
df = pd.read_csv(input_file, header=None)
df[""] = [random.randint(5, 20) for _ in range(len(df))]

# content = ""
# df[""] = content
df.to_csv(input_file, index=False, header=None)

SyntaxError: invalid syntax (449743754.py, line 8)

### **- Remove**

In [4]:
# Delete the last
input_file = "/mnt/c/Users/huyho/Downloads/pack2-groupA/Submission/Done/query-p2-2-qa.csv"
df = pd.read_csv(input_file, header=None)
df = df.iloc[:, :-1]
df.to_csv(input_file, index=False, header=None)